In [1]:
import fiftyone as fo
import fiftyone.utils.coco as fouc
from fiftyone.core.labels import Detection
from PIL import Image

import torch
import torch.nn as nn
import torch.utils
import torch.utils.data

import torchvision
from torchvision.models.segmentation import fcn_resnet50, FCN_ResNet50_Weights
from torchvision.models.detection.mask_rcnn import maskrcnn_resnet50_fpn, MaskRCNNPredictor
from torchvision.models.detection import MaskRCNN_ResNet50_FPN_Weights
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

import matplotlib.pyplot as plt
import numpy as np

import os
import importlib
import sys
from pathlib import Path
from datetime import datetime

import utils
import train
from train import train_one_epoch, evaluate
import transforms as T
importlib.reload(T)

here = os.getcwd()
sys.path.append(os.path.join(here, '..'))
import DatasetLoaders
importlib.reload(DatasetLoaders)
from DatasetLoaders import HRSIDSegmentationDataset
import MaskRCNN_model_wrapper
importlib.reload(MaskRCNN_model_wrapper)
# from MaskRCNN_model_wrapper import MaskRCNNWrapper
from MaskRCNN_model_wrapper import do_training, save_model, get_model
import fiftyone_evaluation as fiftyone_evaluation



In [2]:
# get path to directory above
p = Path()
top_dir = p.absolute().parents[1]
top_dir

PosixPath('/home/ENEE439D_SAR_Ship/SAR-ShipDetection')

In [3]:
name = "HRSID_train"
if name in fo.list_datasets():
    dataset_train = fo.load_dataset(name)
else:
    dataset_dir = top_dir / "Datasets" / "HRSID"
    label_path = top_dir / "Datasets"/ "HRSID" / "annotations" / "train2017.json"
    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset

    dataset_train = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=dataset_type,
        name=name,
        labels_path=label_path
    )

 100% |███████████████| 3642/3642 [25.0s elapsed, 0s remaining, 121.2 samples/s]      


In [4]:
name = "HRSID_test"
if name in fo.list_datasets():
    dataset_test = fo.load_dataset(name)
else:
    dataset_dir = top_dir / "Datasets" / "HRSID"
    label_path = top_dir / "Datasets"/ "HRSID" / "annotations" / "test2017.json"
    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset

    dataset_test = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=dataset_type,
        name=name,
        labels_path=label_path
    )

 100% |███████████████| 1961/1961 [12.8s elapsed, 0s remaining, 91.1 samples/s]       


In [7]:
name = "HRSID_inshore_offshore"
if name in fo.list_datasets():
    dataset_inshore_offshore = fo.load_dataset(name)
else:
    dataset_dir = top_dir / "Datasets" / "HRSID"
    label_path = top_dir / "Datasets"/ "HRSID" / "annotations" / "inshore_offshore.json"
    # The type of the dataset being imported
    dataset_type = fo.types.COCODetectionDataset

    dataset_inshore_offshore = fo.Dataset.from_dir(
        dataset_dir=dataset_dir,
        dataset_type=dataset_type,
        name=name,
        labels_path=label_path
    )

 100% |███████████████| 5604/5604 [36.3s elapsed, 0s remaining, 116.1 samples/s]      


In [5]:
dataset_train.compute_metadata()
dataset_test.compute_metadata()

In [6]:
session = fo.launch_app()

In [7]:
def get_transform_lee():
    transforms = []

    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    transforms.append(T.LeeFilter(5))
    
    return T.Compose(transforms)

In [8]:
model = get_model(2, True)

In [9]:
HRSID_torch_train_lee = HRSIDSegmentationDataset(dataset_train, transforms=get_transform_lee())
HRSID_torch_test_lee = HRSIDSegmentationDataset(dataset_test, transforms=get_transform_lee())

In [ ]:
do_training(model, HRSID_torch_train_lee, HRSID_torch_test_lee, num_epochs=45, step_size=15)

In [ ]:
save_model(model, "maskrcnn_hrsid_pre_lee_e40_lr005_m5_step15.pt")

In [ ]:
fiftyone_evaluation.add_detections(model, HRSID_torch_test_lee, dataset_test, field_name="predictions_lee")

In [ ]:
results = fo.evaluate_detections(
    dataset_test, 
    "predictions_lee", 
    classes=["ship"],
    eval_key="eval", 
    compute_mAP=True,
    gt_field='detections'
)

In [ ]:
results.mAP()

In [ ]:
results.print_report()

In [5]:
def get_transform_HF_crop():
    transforms = []

    transforms.append(T.PILToTensor())
    transforms.append(T.ConvertImageDtype(torch.float))
    transforms.append(T.RandomHorizontalFlip())
    # transforms.append(T.ScaleJitter((600, 600)))
    transforms.append(T.RandomIoUCrop())
    
    return T.Compose(transforms)

In [6]:
model = get_model(2, True)

In [7]:
HRSID_torch_train_HF_crop = HRSIDSegmentationDataset(dataset_train, transforms=get_transform_HF_crop())
HRSID_torch_test_HF_crop = HRSIDSegmentationDataset(dataset_test, transforms=get_transform_HF_crop())

In [8]:
do_training(model, HRSID_torch_train_HF_crop, HRSID_torch_test_HF_crop, num_epochs=20, step_size=10)

Using cuda


/home/ENEE439D_SAR_Ship/SAR-ShipDetection/models/mask_rcnn/../DatasetLoaders.py:71: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343967769/work/torch/csrc/utils/tensor_new.cpp:245.)
  masks=torch.as_tensor(masks, dtype=torch.uint8)
/home/ENEE439D_SAR_Ship/SAR-ShipDetection/models/mask_rcnn/../DatasetLoaders.py:71: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343967769/work/torch/csrc/utils/tensor_new.cpp:245.)
  masks=torch.as_tensor(masks, dtype=torch.uint8)
/home/ENEE439D_SAR_Ship/SAR-ShipDetection/models/mask_rcnn/../DatasetLoaders.py:71: UserWarning: Creating a tensor from a list

Epoch: [0]  [  0/911]  eta: 4:03:14  lr: 0.000010  loss: 3.0646 (3.0646)  loss_classifier: 0.8783 (0.8783)  loss_box_reg: 0.0703 (0.0703)  loss_mask: 1.5484 (1.5484)  loss_objectness: 0.5065 (0.5065)  loss_rpn_box_reg: 0.0611 (0.0611)  time: 16.0203  data: 14.3077  max mem: 6199
Epoch: [0]  [ 10/911]  eta: 0:28:56  lr: 0.000065  loss: 2.5962 (2.8666)  loss_classifier: 0.8281 (0.8178)  loss_box_reg: 0.1016 (0.1187)  loss_mask: 1.4840 (1.4645)  loss_objectness: 0.1321 (0.4119)  loss_rpn_box_reg: 0.0256 (0.0537)  time: 1.9278  data: 1.4168  max mem: 6199
Epoch: [0]  [ 20/911]  eta: 0:19:25  lr: 0.000120  loss: 2.4771 (2.6223)  loss_classifier: 0.6989 (0.6853)  loss_box_reg: 0.1016 (0.1105)  loss_mask: 1.1646 (1.2110)  loss_objectness: 0.0940 (0.5516)  loss_rpn_box_reg: 0.0178 (0.0639)  time: 0.5730  data: 0.1636  max mem: 7504
Epoch: [0]  [ 30/911]  eta: 0:17:13  lr: 0.000175  loss: 1.3243 (2.2367)  loss_classifier: 0.3975 (0.5721)  loss_box_reg: 0.1078 (0.1153)  loss_mask: 0.7201 (1.0241

/home/ENEE439D_SAR_Ship/SAR-ShipDetection/models/mask_rcnn/../DatasetLoaders.py:71: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343967769/work/torch/csrc/utils/tensor_new.cpp:245.)
  masks=torch.as_tensor(masks, dtype=torch.uint8)


creating index...
index created!
Test:  [  0/491]  eta: 0:51:44  model_time: 0.3059 (0.3059)  evaluator_time: 0.1602 (0.1602)  time: 6.3235  data: 5.8479  max mem: 8063
Test:  [100/491]  eta: 0:04:58  model_time: 0.1989 (0.2016)  evaluator_time: 0.0185 (0.0363)  time: 0.7961  data: 0.5169  max mem: 8063
Test:  [200/491]  eta: 0:03:36  model_time: 0.1950 (0.2068)  evaluator_time: 0.0226 (0.0398)  time: 0.4613  data: 0.2251  max mem: 8063
Test:  [300/491]  eta: 0:02:21  model_time: 0.1864 (0.2008)  evaluator_time: 0.0279 (0.0359)  time: 0.8247  data: 0.5874  max mem: 8063
Test:  [400/491]  eta: 0:01:17  model_time: 0.2456 (0.2260)  evaluator_time: 0.0739 (0.0552)  time: 1.6983  data: 1.1825  max mem: 8063
Test:  [490/491]  eta: 0:00:01  model_time: 0.1863 (0.2414)  evaluator_time: 0.0290 (0.0695)  time: 0.4544  data: 0.1423  max mem: 8063
Test: Total time: 0:08:16 (1.0118 s / it)
Averaged stats: model_time: 0.1863 (0.2414)  evaluator_time: 0.0290 (0.0695)
Accumulating evaluation results.

KeyboardInterrupt: 

In [ ]:
save_model(model, "maskrcnn_hrsid_pre_HF_crop_e45_lr005_m5_step15.pt")

In [ ]:
fiftyone_evaluation.add_detections(model, HRSID_torch_test_HF_crop, dataset_test, field_name="predictions_HF_crop")

Using device cuda
   0% ||--------------|    0/1961 [218.6ms elapsed, ? remaining, ? samples/s] 

/home/ENEE439D_SAR_Ship/miniconda3/envs/torch-310-cuda11.7/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


   1% ||--------------|   16/1961 [8.6s elapsed, 17.5m remaining, 2.0 samples/s] 

In [19]:
results = fo.evaluate_detections(
    dataset_test, 
    "predictions_HF_crop", 
    classes=["ship"],
    eval_key="eval", 
    compute_mAP=True,
    gt_field='detections'
)

Evaluating detections...
 100% |███████████████| 1961/1961 [1.7m elapsed, 0s remaining, 19.5 samples/s]       
Performing IoU sweep...


 100% |███████████████| 1961/1961 [47.3s elapsed, 0s remaining, 56.2 samples/s]      


In [ ]:
results.mAP()

In [ ]:
results.print_report()